In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.7MB 66kB/s 
     |████████████████████████████████| 204kB 38.4MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044182 sha256=360b877ef1ab8e5cb619ed7c5b4c456b716769f067b3abe85598e1f8dc55324d
  Stored in directory: /root/.cache/pip/wheels/57/27/4d/ddacf7143f8d5b76c45c61ee2e43d9f8492fc5a8e78ebd7d37
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 35 not upgraded.
Need to get 35.8 MB of archives.
After this 

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

# Organización de Datos 75.06. Segundo Cuatrimestre de 2018. Examen parcial, primera oportunidad

### 1)  (***) Tenemos información sobre recetas en 3 RDD de Spark. 
###Recetas: (id_receta, nombre, tiempo_preparación, dificultad) 
###Ingredientes: (id_ingrediente, nombre) 
###Ingredientes por Receta: (id_receta, id_ingrediente, cantidad) 
###Se pide: 
###a) Obtener el nombre de todas las recetas que tengan Cordero. (7 puntos) 
###b) Calcular la cantidad total de cada ingrediente si queremos hacer todas las recetas con Cordero que sean fáciles. (8 puntos) 

In [22]:
# (id_receta, nombre, tiempo_preparación, dificultad)
recetas = [(1000, 'lemon pie',100,'media'),
           (1001, 'cheese cake',120,'dificil'),
           (1002, 'pan casero de cordero',60,'facil'),
           (1003, 'pionono',85,'media'),
           (1004, 'cordero a la cruz',112,'dificil'),
           (1005, 'tortas fritas',40,'facil'),
           (1006, 'huevo duro',10,'facil'),
           (1007, 'choclo a la manteca',7,'facil'),
           (1008, 'bolonesa',50,'facil'),
           (1009, 'canelones de cordero',56,'dificil'),
           (1010, 'pasta frola',87,'media'),
           (1011, 'berenjenas en escabeche de cordero',101,'facil'),
           (1012, 'asado de cordero',130,'facil')]

# (id_ingrediente, nombre)
ingredientes = [(200, 'leche'),
                (201, 'azucar'),
                (202, 'manteca'),
                (203, 'margarina'),
                (204, 'aceite'),
                (205, 'huevo'),
                (206, 'harina 000'),
                (207, 'harina 0000'),
                (208, 'harina leudante'),
                (209, 'membrillo'),
                (210, 'berenjena'),
                (211, 'limon'),
                (212, 'queso crema'),
                (213, 'frutilla'),
                (214, 'mondongo'),
                (215, 'levadura'),
                (216, 'carne'),
                (217, 'pimienta'),
                (218, 'sal'),
                (219, 'especias varias'),
                (220, 'cordero')]

#(id_receta, id_ingrediente, cantidad)
ingredientes_por_receta = [(1000,212, 400 ),
                           (1001,213, 520 ),
                           (1002,220, 100 ),
                           (1003,206, 200 ),
                           (1004,220, 300 ),
                           (1005,207, 100 ),
                           (1006,205, 140 ),
                           (1007,202, 160 ),
                           (1008,216, 60 ),
                           (1009,220, 40 ),
                           (1010,209, 40 ),
                           (1011,220, 100 ),
                           (1012,220, 140 ),
                           (1004,202, 160 ),
                           (1005,216, 180 ),
                           (1006,211, 60 ),
                           (1007,200, 40 ),
                           (1008,214, 50 ),
                           (1009,204, 40 ),
                           (1010,201, 40 ),
                           (1011,203, 90 ),
                           (1012,219, 40 ),
                           (1004,209, 40 ),
                           (1005,217, 400 ),
                           (1006,212, 500 ),
                           (1007,205, 300 ),
                           (1008,215, 260 ),
                           (1009,210, 100 ),
                           (1010,205, 100 ),
                           (1011,218, 100 ),
                           (1012,218, 400 )
                           ]

In [23]:
recetasRDD = sc.parallelize(recetas)
ingredientesRDD = sc.parallelize(ingredientes)
ingredientes_por_recetaRDD = sc.parallelize(ingredientes_por_receta)

In [24]:
# a) Obtener el nombre de todas las recetas que tengan Cordero. (7 puntos)
# Unifico las RDD's de a dos. Primero, obtengo la relacion codigo-ingrediente
codigo_ingredientes = ((ingredientes_por_recetaRDD.map(lambda x: (x[1],x[0]))).join(ingredientesRDD)).map(lambda x: (x[1][0],x[1][1]))
codigo_ingredientes.collect()

[(1000, 'queso crema'),
 (1006, 'queso crema'),
 (1002, 'cordero'),
 (1004, 'cordero'),
 (1009, 'cordero'),
 (1011, 'cordero'),
 (1012, 'cordero'),
 (1008, 'carne'),
 (1005, 'carne'),
 (1007, 'leche'),
 (1009, 'aceite'),
 (1001, 'frutilla'),
 (1006, 'huevo'),
 (1007, 'huevo'),
 (1010, 'huevo'),
 (1010, 'membrillo'),
 (1004, 'membrillo'),
 (1010, 'azucar'),
 (1005, 'pimienta'),
 (1003, 'harina 000'),
 (1007, 'manteca'),
 (1004, 'manteca'),
 (1008, 'mondongo'),
 (1009, 'berenjena'),
 (1011, 'sal'),
 (1012, 'sal'),
 (1005, 'harina 0000'),
 (1006, 'limon'),
 (1011, 'margarina'),
 (1012, 'especias varias'),
 (1008, 'levadura')]

In [30]:
#unifico los nombres de los ingredientes con las recetas
recetas_cordero = ((recetasRDD.join(codigo_ingredientes)).filter(lambda x: x[1][1]=='cordero')).map(lambda x: (x[0],x[1][0]))
recetas_cordero.collect()


[(1002, 'pan casero de cordero'),
 (1009, 'canelones de cordero'),
 (1004, 'cordero a la cruz'),
 (1011, 'berenjenas en escabeche de cordero'),
 (1012, 'asado de cordero')]

In [37]:
# b) Calcular la cantidad total de cada ingrediente si queremos hacer todas las recetas con Cordero que sean fáciles. (8 puntos)
recetas_cordero_faciles = ((recetasRDD.map(lambda x: (x[0],x[3]))).join(recetas_cordero)).filter(lambda x: x[1][0]=='facil')
recetas_cordero_faciles.collect()

[(1002, ('facil', 'pan casero de cordero')),
 (1011, ('facil', 'berenjenas en escabeche de cordero')),
 (1012, ('facil', 'asado de cordero'))]

In [50]:
#dsd
ingredientes_recetas_cordero_faciles = recetas_cordero_faciles.map(lambda x: (x[0],x[1][1]))
ingredientes_recetas_cordero_faciles.collect()

[(1002, 'pan casero de cordero'),
 (1011, 'berenjenas en escabeche de cordero'),
 (1012, 'asado de cordero')]

In [48]:
ingredientes_cantidades_totales_por_receta = (ingredientes_por_recetaRDD.map(lambda x: (x[0],x[2]))).reduceByKey(lambda x,y: x+y)
ingredientes_cantidades_totales_por_receta.collect()

[(1000, 400),
 (1002, 100),
 (1004, 500),
 (1006, 700),
 (1008, 370),
 (1010, 180),
 (1012, 580),
 (1001, 520),
 (1003, 200),
 (1005, 680),
 (1007, 500),
 (1009, 180),
 (1011, 290)]

In [52]:
ingredientes_cantidades_totales_por_receta_cordero_facil = (ingredientes_recetas_cordero_faciles.join(ingredientes_cantidades_totales_por_receta)).map(lambda x:(x[1][0],x[1][1]))
ingredientes_cantidades_totales_por_receta_cordero_facil.collect()

[('berenjenas en escabeche de cordero', 290),
 ('pan casero de cordero', 100),
 ('asado de cordero', 580)]